In [1]:
#r "nuget: MathNet.Numerics"

open System.Linq;
open System.Collections.Generic
open System.Collections;
open MathNet.Numerics.LinearAlgebra
open MathNet.Numerics.Distributions

Installed Packages MathNet.Numerics, 5.0.0

## Domain

In [2]:
type Goal = 
    | Max
    | Min

type SquaredExponentialKernelParameters = { LengthScale : double; Variance : double }
type DataPoint       = { X : double; Y : double }
type GaussianProcess = 
    { 
        Kernel                   : SquaredExponentialKernelParameters 
        DataPoints               : List<DataPoint>
        mutable CovarianceMatrix : Matrix<double>
    }

type AcquistionFunctionResult = { X : double; Y : double }

type EstimationResult =
    { 
        Mean       : double
        LowerBound : double
        UpperBound : double
        X          : double
    }

type GaussianModel =
    {
        GaussianProcess : GaussianProcess 
        Query           : double -> double
        Inputs          : List<double> 
    }

type ModelResult = 
    { 
        Input                    : List<DataPoint>
        AcquistionFunctionResult : List<AcquistionFunctionResult>
        EstimationResult         : List<EstimationResult>
    }

## Functions

### Kernel 

In [3]:
// Kernel Method.
let gaussianKernelCompute (kernel: SquaredExponentialKernelParameters) (left : double) (right : double) : double  = 
    if left = right then kernel.Variance
    else 
        let squareDistance : double = Math.Pow( left - right, 2 )
        kernel.Variance * Math.Exp( -squareDistance / ( kernel.Variance * kernel.Variance ))

### Estimation 

In [4]:
// Estimation Method.
let estimateAtPoint (gaussianProcess : GaussianProcess) (x : double) : EstimationResult = 
    let kStar : double[] = 
        gaussianProcess.DataPoints
                       .Select(fun dp -> gaussianKernelCompute gaussianProcess.Kernel dp.X dp.Y)
                       .ToArray()
    printfn "kStar: %A" kStar

    let yTrain : double[] = 
        gaussianProcess.DataPoints
                       .Select(fun dp -> dp.Y)
                       .ToArray()

    let ks         : Vector<double> = Vector<double>.Build.Dense kStar
    let f          : Vector<double> = Vector<double>.Build.Dense yTrain
    let common     : Vector<double> = gaussianProcess.CovarianceMatrix.Inverse().Multiply ks
    let mu         : double         = common.DotProduct f
    let confidence : double     = Math.Abs(-common.DotProduct ks + gaussianKernelCompute gaussianProcess.Kernel x x)
    let estimation = { Mean = mu; LowerBound = mu - confidence; UpperBound = mu + confidence; X = x }
    estimation

In [5]:
// Based on each subsequent iteration, we compute the next query point by referring to the surrogate function 
// that produces the highest acquisition value.
let estimateAtRange (gaussianProcess : GaussianProcess) (X : List<double>) : List<EstimationResult> =
    let result = X.Select(fun x -> estimateAtPoint gaussianProcess x).ToList()
    result

### Acquisition

In [6]:
// Acquisition Method.
let expectedImprovement (gaussianProcess : GaussianProcess) 
                        (estimationResult : EstimationResult) 
                        (goal : Goal) : AcquistionFunctionResult = 

    // TODO: Improve this logic by keeping score of the max / min based on the goal.
    let bestValue : DataPoint = 
        match goal with
        | Goal.Max -> gaussianProcess.DataPoints.MaxBy(fun l -> l.Y)
        | Goal.Min -> gaussianProcess.DataPoints.MinBy(fun l -> l.Y)

    let delta : double = estimationResult.Mean - bestValue.Y
    let sigma : double = estimationResult.UpperBound - estimationResult.LowerBound
    let z     : double = delta / sigma
    let next  : double = delta * Normal.CDF(0, 1, z) + sigma * Normal.PDF(0, 1, z)
    { X = estimationResult.X ; Y = Math.Max(next, 0) }

### Model

In [7]:
let addDataPoint (model : GaussianModel) (dataPoint : DataPoint) : unit =

    model.GaussianProcess.DataPoints.Add dataPoint
    let size : int = model.GaussianProcess.DataPoints.Count
    let updatedCovariance : Matrix<double> = Matrix<double>.Build.Dense(size, size) 

    for rowIdx in 0..(model.GaussianProcess.CovarianceMatrix.RowCount - 1) do
        for columnIdx in 0..(model.GaussianProcess.CovarianceMatrix.ColumnCount - 1) do
            updatedCovariance[rowIdx, columnIdx] <- model.GaussianProcess.CovarianceMatrix.[rowIdx, columnIdx]

    for runnerIdx in 0..(size - 1) do
        let value : double = gaussianKernelCompute model.GaussianProcess.Kernel model.GaussianProcess.DataPoints.[runnerIdx].X dataPoint.X
        updatedCovariance[runnerIdx, size - 1] <- value
        updatedCovariance[size - 1, runnerIdx] <- value
    
    updatedCovariance[size - 1, size - 1] <- gaussianKernelCompute model.GaussianProcess.Kernel dataPoint.X dataPoint.X
    updatedCovariance.MapInplace(fun q -> Math.Round(q, 5))
    model.GaussianProcess.CovarianceMatrix <- updatedCovariance

In [8]:
let createModel (gaussianProcess  : GaussianProcess) 
                (query            : double -> double) 
                (min              : double) 
                (max              : double)
                (iterations       : int) : GaussianModel = 

    // Random Uniform Initialization of Inputs.
    let inputs : List<double> = (seq { for i in 0 .. iterations -> 0 }
                                |> Seq.mapi(fun d i -> min + double i * (max - min) / (double iterations - 1.))).ToList()
    { GaussianProcess = gaussianProcess; Query = query; Inputs = inputs }

In [9]:
let findExtrema (gaussianModel : GaussianModel) (goal : Goal) (iterations : int) : ModelResult = 
    addDataPoint gaussianModel { X = gaussianModel.Inputs.[0]; Y = gaussianModel.Query gaussianModel.Inputs.[0] }
    addDataPoint gaussianModel { X = gaussianModel.Inputs.[gaussianModel.Inputs.Count - 1]; Y = gaussianModel.Query gaussianModel.Inputs.[gaussianModel.Inputs.Count - 1] }

    for iterationIdx in 0..iterations do

        // Acquire next data point to explore.
        let nextPointToExplore : double = 
            // Find the data point that maximizes the acquisition function.
            let estimatedAtRange : List<EstimationResult> = estimateAtRange gaussianModel.GaussianProcess gaussianModel.Inputs
            let maxEstimation    : EstimationResult = estimatedAtRange.MaxBy(fun e -> expectedImprovement gaussianModel.GaussianProcess e goal)
            maxEstimation.X

        // Add data point to the model.
        addDataPoint gaussianModel { X = nextPointToExplore; Y = gaussianModel.Query nextPointToExplore }

    let estimationResult : List<EstimationResult> = estimateAtRange gaussianModel.GaussianProcess gaussianModel.Inputs
    {
        Input                    = gaussianModel.GaussianProcess.DataPoints
        AcquistionFunctionResult = estimationResult.Select(fun e -> expectedImprovement gaussianModel.GaussianProcess e goal).ToList() 
        EstimationResult         = estimationResult 
    }

## Tests

In [10]:
let test_model() : GaussianModel =
    let gaussianProcess : GaussianProcess = 
        { 
            Kernel           = { LengthScale = 0.1; Variance = 1 } 
            DataPoints       = List<DataPoint>()
            CovarianceMatrix = Matrix<double>.Build.Dense(1, 1)
        }
    
    let sin (rad : double) : double =
        double ( Math.Sin rad )
    createModel gaussianProcess Math.Sin (2.* -Math.PI) (2. * Math.PI) 10 

In [11]:
let model = test_model()
printfn "%A" model
findExtrema model Goal.Max 10

{ GaussianProcess = { Kernel = { LengthScale = 0.1
                                 Variance = 1.0 }
                      DataPoints = seq []
                      CovarianceMatrix = DenseMatrix 1x1-Double
0
 }
  Query = <fun:test_model@11>
  Inputs = seq [-6.283185307; -6.283185307; -6.283185307; -6.283185307; ...] }
kStar: [|7.157165835e-18; 7.157165835e-18|]
{ Mean = nan
  LowerBound = nan
  UpperBound = nan
  X = -6.283185307 }
kStar: [|7.157165835e-18; 7.157165835e-18|]
{ Mean = nan
  LowerBound = nan
  UpperBound = nan
  X = -6.283185307 }
kStar: [|7.157165835e-18; 7.157165835e-18|]
{ Mean = nan
  LowerBound = nan
  UpperBound = nan
  X = -6.283185307 }
kStar: [|7.157165835e-18; 7.157165835e-18|]
{ Mean = nan
  LowerBound = nan
  UpperBound = nan
  X = -6.283185307 }
kStar: [|7.157165835e-18; 7.157165835e-18|]
{ Mean = nan
  LowerBound = nan
  UpperBound = nan
  X = -6.283185307 }
kStar: [|7.157165835e-18; 7.157165835e-18|]
{ Mean = nan
  LowerBound = nan
  UpperBound = nan
  X 

Input,AcquistionFunctionResult,EstimationResult
List<DataPoint> - X: -6.283185307179586 Y: 2.4492935982947064E-16 - X: -6.283185307179586 Y: 2.4492935982947064E-16 - X: -6.283185307179586 Y: 2.4492935982947064E-16 - X: -6.283185307179586 Y: 2.4492935982947064E-16 - X: -6.283185307179586 Y: 2.4492935982947064E-16 - X: -6.283185307179586 Y: 2.4492935982947064E-16 - X: -6.283185307179586 Y: 2.4492935982947064E-16 - X: -6.283185307179586 Y: 2.4492935982947064E-16 - X: -6.283185307179586 Y: 2.4492935982947064E-16 - X: -6.283185307179586 Y: 2.4492935982947064E-16 - X: -6.283185307179586 Y: 2.4492935982947064E-16 - X: -6.283185307179586 Y: 2.4492935982947064E-16 - X: -6.283185307179586 Y: 2.4492935982947064E-16,List<AcquistionFunctionResult> - X: -6.283185307179586 Y: NaN - X: -6.283185307179586 Y: NaN - X: -6.283185307179586 Y: NaN - X: -6.283185307179586 Y: NaN - X: -6.283185307179586 Y: NaN - X: -6.283185307179586 Y: NaN - X: -6.283185307179586 Y: NaN - X: -6.283185307179586 Y: NaN - X: -6.283185307179586 Y: NaN - X: -6.283185307179586 Y: NaN - X: -6.283185307179586 Y: NaN,List<EstimationResult> - Mean: NaN LowerBound: NaN UpperBound: NaN X: -6.283185307179586 - Mean: NaN LowerBound: NaN UpperBound: NaN X: -6.283185307179586 - Mean: NaN LowerBound: NaN UpperBound: NaN X: -6.283185307179586 - Mean: NaN LowerBound: NaN UpperBound: NaN X: -6.283185307179586 - Mean: NaN LowerBound: NaN UpperBound: NaN X: -6.283185307179586 - Mean: NaN LowerBound: NaN UpperBound: NaN X: -6.283185307179586 - Mean: NaN LowerBound: NaN UpperBound: NaN X: -6.283185307179586 - Mean: NaN LowerBound: NaN UpperBound: NaN X: -6.283185307179586 - Mean: NaN LowerBound: NaN UpperBound: NaN X: -6.283185307179586 - Mean: NaN LowerBound: NaN UpperBound: NaN X: -6.283185307179586 - Mean: NaN LowerBound: NaN UpperBound: NaN X: -6.283185307179586


## References

1. [Gaussian Processes](http://krasserm.github.io/2018/03/19/gaussian-processes/)
2. [Bayesian Optimization From Scratch](https://machinelearningmastery.com/what-is-bayesian-optimization/)

In [12]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.360602+9bf026dabac44a6256a65168fa93dcb7e2edcca4Library version: 1.0.0-beta.22606.2+9bf026dabac44a6256a65168fa93dcb7e2edcca4Build date: 2022-12-10T01:03:09.8221170Zhttps://github.com/dotnet/interactive
